In [1]:
#importing claims data json from s3 to pyspark
sales_df = spark.read.format("csv").load("s3://lhn-capstone/sales.csv",header = "true")

VBox()

Starting Spark application


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
#checking the dataframe
sales_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------------+----------------+-------------+-------+--------------------+-------------+-----------+------------+----+------+
|sales_id|    ad_placed_on|         sold_on|selling_price| Region|               State|         City|seller_type|       owner|sold|car_id|
+--------+----------------+----------------+-------------+-------+--------------------+-------------+-----------+------------+----+------+
|       1|2016-09-04 21:15|            null|       450000|   East|District of Columbia|   Washington| Individual| First Owner|   N|123477|
|       2|2016-09-05 00:15|            null|       370000|   East|            New York|New York City| Individual|Second Owner|   N|123478|
|       3|2016-09-13 15:24|            null|       158000|Central|            Illinois|      Chicago| Individual| Third Owner|   N|123479|
|       4|2016-09-15 12:16|2016-11-09 07:47|       225000|Central|            Illinois|      Chicago| Individual| First Owner|   Y|123480|
|       5|2016-10-02 22:07|

In [3]:
#creating new duplicate claims df
sales_df_1=sales_df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#converting the sold column to timestamp to extract month of column
from pyspark.sql.functions import to_timestamp,date_format
from pyspark.sql.functions import col

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
sales_df_1=sales_df_1.withColumn("sold_on",to_timestamp(col("sold_on"))).withColumn("Month", date_format(col("sold_on"), "M"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
sales_df_1.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------------+-------------------+-------------+-------+--------------------+-------------+-----------+------------+----+------+-----+
|sales_id|    ad_placed_on|            sold_on|selling_price| Region|               State|         City|seller_type|       owner|sold|car_id|Month|
+--------+----------------+-------------------+-------------+-------+--------------------+-------------+-----------+------------+----+------+-----+
|       1|2016-09-04 21:15|               null|       450000|   East|District of Columbia|   Washington| Individual| First Owner|   N|123477| null|
|       2|2016-09-05 00:15|               null|       370000|   East|            New York|New York City| Individual|Second Owner|   N|123478| null|
|       3|2016-09-13 15:24|               null|       158000|Central|            Illinois|      Chicago| Individual| Third Owner|   N|123479| null|
|       4|2016-09-15 12:16|2016-11-09 07:47:00|       225000|Central|            Illinois|      Chicago| Individ

In [7]:

from pyspark.sql.functions import year,to_timestamp
sales_df_2 = sales_df_1.withColumn('Year',year(to_timestamp('sold_on', 'yyyy-MM-dd hh:mm')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
sales_df_2.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------------+-------------------+-------------+-------+--------------------+-------------+-----------+------------+----+------+-----+----+
|sales_id|    ad_placed_on|            sold_on|selling_price| Region|               State|         City|seller_type|       owner|sold|car_id|Month|Year|
+--------+----------------+-------------------+-------------+-------+--------------------+-------------+-----------+------------+----+------+-----+----+
|       1|2016-09-04 21:15|               null|       450000|   East|District of Columbia|   Washington| Individual| First Owner|   N|123477| null|null|
|       2|2016-09-05 00:15|               null|       370000|   East|            New York|New York City| Individual|Second Owner|   N|123478| null|null|
|       3|2016-09-13 15:24|               null|       158000|Central|            Illinois|      Chicago| Individual| Third Owner|   N|123479| null|null|
|       4|2016-09-15 12:16|2016-11-09 07:47:00|       225000|Central|            I

In [9]:
sales_df_2 = sales_df_2.withColumn("selling_price",sales_df_2["selling_price"].cast('float'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
from pyspark.sql import functions as sf
metric1=sales_df_2.groupBy('Year','Month').agg(sf.sum('selling_price').alias('Total_Sales_Amount'), sf.count('sales_id').alias('Total_Cars_Sold')).sort('Year','Month').dropna()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
metric1.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+------------------+---------------+
|Year|Month|Total_Sales_Amount|Total_Cars_Sold|
+----+-----+------------------+---------------+
|2016|   10|      1.68808993E8|            208|
|2016|   11|       4.2828995E7|             60|
|2016|   12|         4420000.0|              4|
|2017|    1|      1.97461987E8|            283|
|2017|    2|      8.22323097E8|           1351|
|2017|    3|     1.596319859E9|           2382|
|2017|    4|     1.150971531E9|           1849|
|2017|    5|      8.06006652E8|           1266|
|2017|    6|         6554999.0|             15|
|2017|    7|          400000.0|              1|
|2017|    8|          600000.0|              1|
|2017|    9|          1.1009E7|             18|
+----+-----+------------------+---------------+

In [12]:

metric1.write.parquet("s3://lhn-capstone/metric_sales.parquet")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…